In [26]:
import osmnx as ox
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import streamlit as st
from streamlit_folium import folium_static
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')

In [18]:
#df_new= pd.read_excel('df_copia_seguridad30-04-2022.xlsx') #20 columnas, con las de mercator
df_new= pd.read_excel("prueba_colores.xlsx")


In [ ]:
df_acum= pd.read_excel('df_definitivo.xlsx')

In [24]:
#poner aqui la funcio buena. esta no sirve en el mapa

#geolocator = Nominatim(user_agent="My-App")

#location = geolocator.geocode("Calle Juan Ramon jimenez, 15, Madrid") 

#print((location.latitude, location.longitude))

In [25]:
#type(location)

In [26]:
#diccionario={'REPSOL': 'REPSOL.png', 'CEPSA': 'CEPSA.png'}
#diccionario


In [22]:
df_new = df_new.sort_values(by=['Distance'])
df_new=df_new.head(10)
df_new

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish,Distance
6386,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5",-3.662056,40.548306,"1,919",No disponible,"2,069","1,959","2,029",GALP,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513816.776449839 -407936.0483483151),447.131891
6389,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE MARQUES DE LA VALDAVIA (DEL), 138",-3.653944,40.546944,"1,909",No disponible,"2,051","1,969","2,059",BP VALDAVIA,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513665.159303379 -407031.1811113249),516.618744
6398,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA MARQUES DE LA VALDAVIA, 132",-3.651861,40.546556,"1,819",No disponible,No disponible,"1,819",No disponible,PETROPRIX,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513621.967340952 -406798.8305516534),745.354730
6387,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE VALGRANDE, 30",-3.654139,40.539583,"1,819",No disponible,No disponible,"1,819","1,839",BALLENOIL,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512845.73653165 -407052.9326315577),901.605721
6399,MADRID,Alcobendas,ALCOBENDAS,28108,"CALLE VALGRANDE, 11",-3.649750,40.536389,"1,819",No disponible,No disponible,"1,819",No disponible,PETROPRIX,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512490.182078056 -406563.3572507771),1482.389399
6401,MADRID,Alcobendas,ALCOBENDAS,28108,"CALLE FRANCISCO GERVAS, 13",-3.652250,40.533722,"1,899",No disponible,"2,049","1,939","2,039",REPSOL,L-V: 06:00-22:00; S: 07:00-14:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512193.292996109 -406842.2219498365),1573.731913
6391,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA MONTE DE VALDELATAS, 5",-3.659667,40.528722,"1,909",No disponible,"2,051","1,969","2,059",BP EL SAPO-ALCOBENDAS,L-D: 06:00-13:59 y 14:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511636.695542145 -407669.5623112876),1969.246769
6396,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA, 3",-3.643194,40.537250,"1,859",No disponible,"1,999","1,899","1,979",SHELL,L-S: 06:00-22:00; D: 09:00-17:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512586.02815963 -405832.0661490023),1990.599349
6395,MADRID,Alcobendas,ALCOBENDAS,28100,"CARRETERA M-603 KM. 14,900",-3.651583,40.528806,"1,859",No disponible,"1,999","1,899","1,979",SHELL,L-D: 06:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511646.046379371 -406767.8207721841),2104.234713
6397,MADRID,Alcobendas,ALCOBENDAS,28100,"CARRETERA A-1 KM. 14,500",-3.649417,40.528194,"1,899",No disponible,"2,039","1,939","1,969",CARREFOUR,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511577.918851005 -406526.2125314499),2265.466971


In [9]:
mapa_loop = folium.Map(location=[40.546376,-3.638541], zoom_start=15)

#marker_cluster = MarkerCluster().add_to(mapa_loop)

def ubi_gasolinera(df_new):
    for i in range(0,len(df_new)):
        html=f"""
            <div style="font-family: arial; color: black">
            <h><b> {df_new.iloc[i]['Rótulo']}</h></b>
            <p>Prices:</p>
                <li> Gasolina 95:{df_new.iloc[i]['Precio gasolina 95 E5']} €</li>
                <li> Diésel:{df_new.iloc[i]['Precio gasóleo A']} €</li>
            <p><b>Dirección:</b></p>
            <p>{df_new.iloc[i]['Dirección']}</p>
            </ul> 
            </p>
            <img src="my_plot_name.png">
            <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p></div>
            """
        
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        
        folium.Marker(
                location=[df_new.iloc[i]['Latitud'],df_new.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="green",  icon="ok-sign"),
        ).add_to(mapa_loop)
        
    return mapa_loop

def ubi_actual(df_new):
    html=f"""
        <div style="font-family: times new roman; color: green">
        <h><b> Actualmente te encuentras aquí </b></h></div>
        """
    iframe = folium.IFrame(html=html, width=150, height=60)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[df_new.iloc[0]['Latitud_partida'],df_new.iloc[0]['Longitud_partida']],
        popup=popup,
        icon=folium.Icon(color="cadetblue",  icon="home"),
).add_to(mapa_loop)  
    return mapa_loop
    
#ubi_gasolinera(df_new)
#ubi_actual(df_new)

In [68]:
df_grafico=[]
direccion_gasolineras=list(df_new['Dirección'].unique())
for i in direccion_gasolineras:
    df_grafico.append(df_acum[df_acum.Dirección.isin([i])])
df_grafico[0]

#fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(13,5))
#x= df_grafico[0]['Fecha de extracción']
#y= df_grafico[0] ['Precio gasolina 95 E5'].sort_values()

#ax.plot(x,y)
#plt.show()
#fig.savefig("my_plot_name.png")

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción
6356,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,809",No disponible,"1,959","1,849","1,919",GALP,L-D: 24H,04-04-2022
17982,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,799",No disponible,"1,949","1,839","1,909",GALP,L-D: 24H,05-04-2022
29623,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,809",No disponible,"1,959","1,819","1,889",GALP,L-D: 24H,09-04-2022
41269,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,809",No disponible,"1,959","1,819","1,889",GALP,L-D: 24H,10-04-2022
52937,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,819",No disponible,"1,969","1,829","1,899",GALP,L-D: 24H,11-04-2022
64620,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,809",No disponible,"1,959","1,819","1,889",GALP,L-D: 24H,12-04-2022
76317,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,819",No disponible,"1,969","1,839","1,909",GALP,L-D: 24H,14-04-2022
88007,MADRID,ALCOBENDAS,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,839",No disponible,"1,989","1,859","1,929",GALP,L-D: 24H,15-04-2022
99678,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,839",No disponible,"1,989","1,859","1,929",GALP,L-D: 24H,16-04-2022
111335,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5","-3,662056","40,548306","1,849",No disponible,"1,999","1,879","1,949",GALP,L-D: 24H,17-04-2022


In [69]:
# introducir imagen con html para meter el grafico en la etiqueta.
#https://python-graph-gallery.com/312-add-markers-on-folium-map

In [70]:
#st.title('Map of Gas')

In [71]:
#folium_static(mapa_loop)

In [72]:
#mapa_loop.py

In [20]:
def statistical_quartiles(dataset, column, percentile):
    lst = dataset[column].tolist()
    data = sorted(lst)
    if percentile != 100:
        index = int((len(data))*percentile/100)
        #Par, si el resto es 0.  
        if len(lst) % 2 != 0:
            return data[index]

        #Impar, si el resto != de 0.
        else:
            return (data[index - 1] + data[index]) / 2
        
    elif percentile == 100:
        index = int(len(data))-1  
        return data[index]
    
#df_colores = df_new.iloc[:,[7,10]]
#df_colores

In [21]:
df_colores = df_new.iloc[:,[7,10]]
df_colores

,Precio gasolina 95 E5,Precio gasóleo A
6386,"1,919","1,959"
6389,"1,909","1,969"
6398,"1,819","1,819"
6387,"1,819","1,819"
6399,"1,819","1,819"
6401,"1,899","1,939"
6391,"1,909","1,969"
6396,"1,859","1,899"
6395,"1,859","1,899"
6397,"1,899","1,939"


In [8]:
df_colores = df_new.iloc[:,[7,10]]
df_colores['Precio gasolina 95 E5'] = df_colores['Precio gasolina 95 E5'].apply(lambda x: x.replace(',','.'))
df_colores['Precio gasóleo A'] = df_colores['Precio gasóleo A'].apply(lambda x: x.replace(',','.'))
df_colores['Precio gasolina 95 E5']=df_colores['Precio gasolina 95 E5'].astype('float64')
df_colores['Precio gasóleo A']=df_colores['Precio gasóleo A'].astype('float64')


percentil_50=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 50) #amarillo
percentil_75=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 75) #rojo
percentil_25=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 25) #verde


In [9]:
df_colores

,Precio gasolina 95 E5,Precio gasóleo A
6383,1.799,1.799
6394,1.889,1.949
6391,1.909,1.939
6385,1.859,1.899
6389,1.799,1.799
6401,1.799,1.799
6400,1.899,1.939
6399,1.899,1.939
6397,1.829,1.869
6386,1.919,1.959


In [10]:

df_new['Precio gasolina 95 E5'] = df_new['Precio gasolina 95 E5'].apply(lambda x: x.replace(',','.'))
df_new['Precio gasóleo A'] = df_new['Precio gasóleo A'].apply(lambda x: x.replace(',','.'))

df_new['Precio gasóleo A']=df_new['Precio gasóleo A'].astype('float64')
df_new['Precio gasolina 95 E5']=df_new['Precio gasolina 95 E5'].astype('float64')


In [11]:
bins = [0,percentil_25,percentil_75,100]
colores = ['verde','amarillo','rojo']
df_new['Colores']= pd.cut(df_new['Precio gasolina 95 E5'],bins,labels = colores)

In [12]:
df_new

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,...,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish,Distance,Colores
6383,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA MARQUES DE LA VALDAVIA, 132",-3.651861,40.546556,1.799,No disponible,No disponible,...,No disponible,PETROPRIX,L-D: 24H,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4513621.967340952 -406798.8305516534),856.486704,verde
6394,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE MARQUES DE LA VALDAVIA (DEL), 138",-3.653944,40.546944,1.889,No disponible,"2,031",...,"2,039",BP VALDAVIA,L-D: 24H,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4513665.159303379 -407031.1811113249),1066.535964,amarillo
6391,MADRID,Alcobendas,ALCOBENDAS,28100,"CL AVENIDA DE ESPAÑA, S.N.",-3.634639,40.546167,1.909,No disponible,"2,069",...,"2,019",REPSOL,L-D: 24H,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4513578.664059033 -404877.8039682711),1167.296308,rojo
6385,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA, 3",-3.643194,40.537250,1.859,No disponible,"1,999",...,"1,979",SHELL,L-S: 06:00-22:00; D: 09:00-17:00,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4512586.02815963 -405832.0661490023),1341.234094,amarillo
6389,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE VALGRANDE, 30",-3.654139,40.539583,1.799,No disponible,No disponible,...,"1,819",BALLENOIL,L-D: 24H,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4512845.73653165 -407052.9326315577),1505.920624,verde
6401,MADRID,Alcobendas,ALCOBENDAS,28108,"CALLE VALGRANDE, 11",-3.649750,40.536389,1.799,No disponible,No disponible,...,No disponible,PETROPRIX,L-D: 24H,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4512490.182078056 -406563.3572507771),1536.210811,verde
6400,MADRID,Alcobendas,ALCOBENDAS,28108,"BULEVAR SALVADOR ALLENDE (DE), 13",-3.634389,40.537750,1.899,No disponible,"2,049",...,"2,019",REPSOL,L-S: 06:00-23:00; D: 07:00-23:00,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4512641.687905027 -404849.9180091961),1714.132946,amarillo
6399,MADRID,Alcobendas,ALCOBENDAS,28108,"CL BOULEVARD SALVADOR ALLENDE, 12-1",-3.633944,40.537500,1.899,"1,939",No disponible,...,"2,019",REPSOL,L-D: 06:00-22:00,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4512613.858032328 -404800.2810211646),1768.094584,amarillo
6397,MADRID,Alcobendas,ALCOBENDAS,28108,"AVENIDA OLIMPICA, 9",-3.636806,40.535194,1.829,No disponible,"1,969",...,"1,899",ALCAMPO,L-D: 24H,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4512357.155286557 -405119.5197854434),1788.981347,amarillo
6386,MADRID,Alcobendas,ALCOBENDAS,28100,"CARRETERA ANTIGUA NACIONAL I KM. 15,5",-3.642250,40.532806,1.919,No disponible,"2,069",...,"2,029",GALP,L-D: 07:00-23:00,29-04-2022,40.549209,-3.644654,POINT (4513917.342477823 -405994.8883876073),POINT (4512091.324342543 -405726.7678082834),1845.597701,rojo


In [14]:
#funcion buena para el pipeline. está copiada abajo.

mapa_loop = folium.Map(location=[40.546376,-3.638541], zoom_start=15)

#marker_cluster = MarkerCluster().add_to(mapa_loop)

def ubi_gasolinera(df_new):
    for i in range(0,len(df_new)):
        html=f"""
            <div style="font-family: arial; color: black">
            <h><b> {df_new.iloc[i]['Rótulo']}</h></b>
            <p>Prices:</p>
                <li> Gasolina 95:{df_new.iloc[i]['Precio gasolina 95 E5']} €</li>
                <li> Diésel:{df_new.iloc[i]['Precio gasóleo A']} €</li>
            <p><b>Dirección:</b></p>
            <p>{df_new.iloc[i]['Dirección']}</p>
            </ul> 
            </p>
            <img src="my_plot_name.png">
            <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p></div>
            """
        
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        
        if df_new.iloc[i]['Colores']== 'verde':
            folium.Marker(
                location=[df_new.iloc[i]['Latitud'],df_new.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="green",  icon="ok-sign"),
            ).add_to(mapa_loop)
        elif df_new.iloc[i]['Colores']== 'amarillo':
            folium.Marker(
                location=[df_new.iloc[i]['Latitud'],df_new.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="orange",  icon="ok-sign"),
            ).add_to(mapa_loop)
            
        elif df_new.iloc[i]['Colores']== 'rojo':
            folium.Marker(
                location=[df_new.iloc[i]['Latitud'],df_new.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="red",  icon="ok-sign"),
            ).add_to(mapa_loop)
            
    return mapa_loop

def ubi_actual(df_new):
    html=f"""
        <div style="font-family: times new roman; color: green">
        <h><b> Actualmente te encuentras aquí </b></h></div>
        """
    iframe = folium.IFrame(html=html, width=150, height=60)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[df_new.iloc[0]['Latitud_partida'],df_new.iloc[0]['Longitud_partida']],
        popup=popup,
        icon=folium.Icon(color="cadetblue",  icon="home"),
).add_to(mapa_loop)  
    return mapa_loop
    
ubi_gasolinera(df_new)
ubi_actual(df_new)

## funciones para pipeline

In [41]:
def statistical_quartiles(dataset, column, percentile):
    lst = dataset[column].tolist()
    data = sorted(lst)
    if percentile != 100:
        index = int((len(data))*percentile/100)
        #Par, si el resto es 0.  
        if len(lst) % 2 != 0:
            return data[index]
        else:
            return (data[index - 1] + data[index]) / 2
    elif percentile == 100:
        index = int(len(data))-1  
        return data[index]
    
#df_colores = df_new.iloc[:,[7,10]]

In [42]:
def colores():
    df_colores = df_new.iloc[:,[7,10]]
    df_colores['Precio gasolina 95 E5'] = df_colores['Precio gasolina 95 E5'].apply(lambda x: x.replace(',','.'))
    df_colores['Precio gasóleo A'] = df_colores['Precio gasóleo A'].apply(lambda x: x.replace(',','.'))
    df_colores['Precio gasolina 95 E5']=df_colores['Precio gasolina 95 E5'].astype('float64')
    df_colores['Precio gasóleo A']=df_colores['Precio gasóleo A'].astype('float64')
    percentil_50=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 50) #amarillo
    percentil_75=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 75) #rojo
    percentil_25=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 25) #verde  
    df_new['Precio gasolina 95 E5'] = df_new['Precio gasolina 95 E5'].apply(lambda x: x.replace(',','.'))
    df_new['Precio gasóleo A'] = df_new['Precio gasóleo A'].apply(lambda x: x.replace(',','.'))
    df_new['Precio gasóleo A']=df_new['Precio gasóleo A'].astype('float64')
    df_new['Precio gasolina 95 E5']=df_new['Precio gasolina 95 E5'].astype('float64')
    bins = [0,percentil_25,percentil_75,100]
    colores = ['verde','amarillo','rojo']
    df_new['Colores']= pd.cut(df_new['Precio gasolina 95 E5'],bins,labels = colores)
    return df_new


In [43]:
df_new = colores()


In [44]:
df_new

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,...,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish,Distance,Colores
6386,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5",-3.662056,40.548306,1.919,No disponible,"2,069",...,"2,029",GALP,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513816.776449839 -407936.0483483151),447.131891,rojo
6389,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE MARQUES DE LA VALDAVIA (DEL), 138",-3.653944,40.546944,1.909,No disponible,"2,051",...,"2,059",BP VALDAVIA,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513665.159303379 -407031.1811113249),516.618744,rojo
6398,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA MARQUES DE LA VALDAVIA, 132",-3.651861,40.546556,1.819,No disponible,No disponible,...,No disponible,PETROPRIX,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513621.967340952 -406798.8305516534),745.354730,verde
6387,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE VALGRANDE, 30",-3.654139,40.539583,1.819,No disponible,No disponible,...,"1,839",BALLENOIL,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512845.73653165 -407052.9326315577),901.605721,verde
6399,MADRID,Alcobendas,ALCOBENDAS,28108,"CALLE VALGRANDE, 11",-3.649750,40.536389,1.819,No disponible,No disponible,...,No disponible,PETROPRIX,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512490.182078056 -406563.3572507771),1482.389399,verde
6401,MADRID,Alcobendas,ALCOBENDAS,28108,"CALLE FRANCISCO GERVAS, 13",-3.652250,40.533722,1.899,No disponible,"2,049",...,"2,039",REPSOL,L-V: 06:00-22:00; S: 07:00-14:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512193.292996109 -406842.2219498365),1573.731913,amarillo
6391,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA MONTE DE VALDELATAS, 5",-3.659667,40.528722,1.909,No disponible,"2,051",...,"2,059",BP EL SAPO-ALCOBENDAS,L-D: 06:00-13:59 y 14:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511636.695542145 -407669.5623112876),1969.246769,rojo
6396,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA, 3",-3.643194,40.537250,1.859,No disponible,"1,999",...,"1,979",SHELL,L-S: 06:00-22:00; D: 09:00-17:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512586.02815963 -405832.0661490023),1990.599349,amarillo
6395,MADRID,Alcobendas,ALCOBENDAS,28100,"CARRETERA M-603 KM. 14,900",-3.651583,40.528806,1.859,No disponible,"1,999",...,"1,979",SHELL,L-D: 06:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511646.046379371 -406767.8207721841),2104.234713,amarillo
6397,MADRID,Alcobendas,ALCOBENDAS,28100,"CARRETERA A-1 KM. 14,500",-3.649417,40.528194,1.899,No disponible,"2,039",...,"1,969",CARREFOUR,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511577.918851005 -406526.2125314499),2265.466971,amarillo


In [45]:


#marker_cluster = MarkerCluster().add_to(mapa_loop)

def ubi_gasolinera(df_new):
    mapa_loop = folium.Map(location=[40.546376,-3.638541], zoom_start=15)
    for i in range(0,len(df_new)):
        html=f"""
            <div style="font-family: arial; color: black">
            <h><b> {df_new.iloc[i]['Rótulo']}</h></b>
            <p>Prices:</p>
                <li> Gasolina 95:{df_new.iloc[i]['Precio gasolina 95 E5']} €</li>
                <li> Diésel:{df_new.iloc[i]['Precio gasóleo A']} €</li>
            <p><b>Dirección:</b></p>
            <p>{df_new.iloc[i]['Dirección']}</p>
            </ul> 
            </p>
            <img src="my_plot_name.png">
            <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p></div>
            """
        
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        
        if df_new.iloc[i]['Colores']== 'verde':
            folium.Marker(
                location=[df_new.iloc[i]['Latitud'],df_new.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="green",  icon="ok-sign"),
            ).add_to(mapa_loop)
        elif df_new.iloc[i]['Colores']== 'amarillo':
            folium.Marker(
                location=[df_new.iloc[i]['Latitud'],df_new.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="orange",  icon="ok-sign"),
            ).add_to(mapa_loop)
            
        elif df_new.iloc[i]['Colores']== 'rojo':
            folium.Marker(
                location=[df_new.iloc[i]['Latitud'],df_new.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="red",  icon="ok-sign"),
            ).add_to(mapa_loop)
    mapa_loop.save('map.html')
    return mapa_loop


def ubi_actual(df_new):
    html=f"""
        <div style="font-family: times new roman; color: green">
        <h><b> Actualmente te encuentras aquí </b></h></div>
        """
    iframe = folium.IFrame(html=html, width=150, height=60)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[df_new.iloc[0]['Latitud_partida'],df_new.iloc[0]['Longitud_partida']],
        popup=popup,
        icon=folium.Icon(color="cadetblue",  icon="home"),
).add_to(mapa_loop)
    mapa_loop.save('map.html')
    return mapa_loop
    


In [46]:
mapa_loop = ubi_gasolinera(df_new)
mapa_loop = ubi_actual(df_new)

In [47]:
mapa_loop

In [32]:
def ubi_actual(df_new):
    mapa_loop = folium.Map(location=[40.546376,-3.638541], zoom_start=15)
    html=f"""
        <div style="font-family: times new roman; color: green">
        <h><b> Actualmente te encuentras aquí </b></h></div>
        """
    iframe = folium.IFrame(html=html, width=150, height=60)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[df_new.iloc[0]['Latitud_partida'],df_new.iloc[0]['Longitud_partida']],
        popup=popup,
        icon=folium.Icon(color="cadetblue",  icon="home"),
).add_to(mapa_loop) 
    mapa_loop.save('map.html')
    return mapa_loop

In [33]:
mapa_loop = ubi_actual(df_new)

In [34]:
mapa_loop